In [59]:
import pandas as pd 
import numpy as np
import requests
import io
import csv
from path import Path
import psycopg2
from config import db_password

In [60]:
# Connction to AWS nba database
connection = psycopg2.connect(
    host = 'nba-db.cpmpsi1pfaz0.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = db_password,
    database='postgres'
    )
cursor=connection.cursor()

In [73]:
#eventually we will replace this with a database
#url = 'https://github.com/AForbis/Bootcamp_Final_Project/blob/datasourcing/SeasonData/AllNBA_Unpivoted.xlsx'
##url= Path("Resources/AllNBA_UNpivoted.xlsx")
#load in All NBA teams, drop the league column (we are not including the ABA, I already deleted all that from the file)
##All_NBA_Unpivoted_df = pd.DataFrame(pd.read_excel(url)).drop(['Lg'],axis=1)
#rename the players column
##All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
##All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
##All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
#All_NBA_Unpivoted_df.head(10)

In [61]:
#read in data from database for All NBA table
nba_unpivoted_sql = 'SELECT * FROM public.allnba_unpivoted'
All_NBA_Unpivoted_df = pd.read_sql(nba_unpivoted_sql, con=connection).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
#All_NBA_Unpivoted_df.head()

In [62]:
#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

#make dummies for position and team (1st, 2nd, 3rd)
All_NBA_Unpivoted_df = pd.get_dummies(All_NBA_Unpivoted_df,columns=['Position','Tm']).drop(['Season','PlayerName'],axis=1)
#rename the columns (cosmetic)
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_C':'AllNBA_Center'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_F':'AllNBA_Forward'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_G':'AllNBA_Guard'})
#All_NBA_Unpivoted_df.head(10)

In [63]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
#AllNBAPlayersList

In [76]:
#url = 'SeasonData/Seasons_Stats.xlsx'
##url= Path("Resources/Seasons_Stats.xlsx")
#load in season stats
##season_stats_df = pd.DataFrame(pd.read_excel(url))
##season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
##season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
##season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
#season_stats_df.head()
#season_stats_df.to_csv('ML_DF.csv')

In [64]:
#read in data from database for season stats table
seasons_stats_sql = 'SELECT * FROM public.seasons_stats'
season_stats_df = pd.read_sql(seasons_stats_sql, con=connection)
#divid year into season start and end
season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
season_stats_df['Year_season_end'] = pd.to_numeric(season_stats_df['Year_season_end'])
season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
#season_stats_df.head()

In [65]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
#AllPlayersList_df.to_csv('listofplayers_test.csv')

In [66]:
#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic
Jayson Tatum
Ben Simmons


In [67]:
#merge allNBA status into season stats
Season_Stats_ML_DF = pd.merge(season_stats_df,All_NBA_Unpivoted_df,how='left',left_on = ['Player','Year_season_start'], right_on = ['PlayerName_Formatted','Season_Start_Year'])

In [68]:
# get rid of duplicate cols
Season_Stats_ML_DF = Season_Stats_ML_DF.drop(['Season_Start_Year','PlayerName_Formatted'],axis=1)

In [69]:
# fill in zeros for players that arent all nba
Season_Stats_ML_DF['AllNBA_Center'] = Season_Stats_ML_DF['AllNBA_Center'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Forward'] = Season_Stats_ML_DF['AllNBA_Forward'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Guard'] = Season_Stats_ML_DF['AllNBA_Guard'].fillna(int(0))
Season_Stats_ML_DF['Tm_1st'] = Season_Stats_ML_DF['Tm_1st'].fillna(int(0))
Season_Stats_ML_DF['Tm_2nd'] = Season_Stats_ML_DF['Tm_2nd'].fillna(int(0))
Season_Stats_ML_DF['Tm_3rd'] = Season_Stats_ML_DF['Tm_3rd'].fillna(int(0))


In [70]:
#get list of positions
Positions_List = Season_Stats_ML_DF['Pos'].unique()


In [71]:
# create functions to determine if player is a guard forward and/or center
def is_forward(pos):
    possible_pos = [ 'SF',  'PF', 'PF-C', 'SG-SF', 'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF']
    if pos in possible_pos: isF = 1
    else: isF = 0
    return isF

def is_guard(pos):
    possible_pos = ['PG','SG','SG-SF','PG-SG','SG-PG','SF-SG','SG-PF','PG-SF']
    if pos in possible_pos: isG = 1
    else: isG = 0
    return isG

def is_center(pos):
    possible_pos = ['C','PF-C','C-PF','C-SF']
    if pos in possible_pos: isC = 1
    else: isC = 0
    return isC

In [72]:
#Create dummy columns for G, F, C
Season_Stats_ML_DF['is_Guard'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_guard(pos) )
Season_Stats_ML_DF['is_Forward'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_forward(pos) )
Season_Stats_ML_DF['is_Center'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_center(pos) )


In [73]:
#Drop the Pos Column since we have dummy columns
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Pos'],axis=1)


In [74]:
#make dummies for team
Season_Stats_ML_DF = pd.get_dummies(Season_Stats_ML_DF,columns=['Tm','Year_season_start'])


In [75]:
# drop year end and player name
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_end','Player'],axis=1)
Season_Stats_ML_DF.head()

,Age,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,...,Year_season_start_2007,Year_season_start_2008,Year_season_start_2009,Year_season_start_2010,Year_season_start_2011,Year_season_start_2012,Year_season_start_2013,Year_season_start_2014,Year_season_start_2015,Year_season_start_2016
0,31,16,0,158.0,11.4,0.449,NaN,0.383,12.4,20.9,...,0,0,0,0,0,0,0,0,0,0
1,31,2,0,24.0,11.5,0.279,NaN,0.231,26.0,37.3,...,0,0,0,0,0,0,0,0,0,0
2,23,31,0,285.0,7.6,0.367,NaN,0.556,16.3,15.4,...,0,0,0,0,0,0,0,0,0,0
3,30,77,0,988.0,14.0,0.551,NaN,0.619,1.2,7.3,...,0,0,0,0,0,0,0,0,0,0
4,29,82,0,2186.0,13.6,0.484,NaN,0.238,3.5,5.2,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#Season_Stats_ML_DF.to_csv(r'C:\Users\18323\Desktop\Classwork\Final_Project\export_dataframe.csv', index = False, header=True)

In [76]:
#Creating Per Game Statistics

#Points per Game
Season_Stats_ML_DF["PPG"] = Season_Stats_ML_DF["PTS"]/Season_Stats_ML_DF["G"]
#Assits Per Game
Season_Stats_ML_DF["APG"] = Season_Stats_ML_DF["AST"]/Season_Stats_ML_DF["G"]
#Rebounds Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["TRB"]/Season_Stats_ML_DF["G"]
#Blocks Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["BLK"]/Season_Stats_ML_DF["G"]
#STLs Per Game
Season_Stats_ML_DF["SPG"] = Season_Stats_ML_DF["STL"]/Season_Stats_ML_DF["G"]

In [97]:
#machine learning part



#Here is the code for how to "Replace NaN with zero and infinity with large finite numbers." using numpy.nan_to_num.

Season_Stats_ML_DF[:] = np.nan_to_num(Season_Stats_ML_DF, copy=False)

#X = Season_Stats_ML_DF_loans.copy()
X = Season_Stats_ML_DF.drop("AllNBA_Center", axis=1)
X.head()      
                                                                        

,Age,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,...,Year_season_start_2011,Year_season_start_2012,Year_season_start_2013,Year_season_start_2014,Year_season_start_2015,Year_season_start_2016,PPG,APG,RPG,SPG
0,31,16,0,158.0,11.4,0.449,NaN,0.383,12.4,20.9,...,0,0,0,0,0,0,3.937500,0.625000,0.187500,0.187500
1,31,2,0,24.0,11.5,0.279,NaN,0.231,26.0,37.3,...,0,0,0,0,0,0,4.000000,1.500000,0.500000,0.500000
2,23,31,0,285.0,7.6,0.367,NaN,0.556,16.3,15.4,...,0,0,0,0,0,0,2.387097,0.709677,0.225806,0.354839
3,30,77,0,988.0,14.0,0.551,NaN,0.619,1.2,7.3,...,0,0,0,0,0,0,6.064935,1.922078,0.064935,0.597403
4,29,82,0,2186.0,13.6,0.484,NaN,0.238,3.5,5.2,...,0,0,0,0,0,0,12.329268,2.743902,0.231707,0.829268


In [98]:
y = Season_Stats_ML_DF["AllNBA_Center"].values.reshape(-1, 1)
y[:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [99]:
# Splitting into Train and Test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [100]:
from sklearn.preprocessing import StandardScaler
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [101]:
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)
predictions



# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [111]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4868,12
Actual 1,13,19


Accuracy Score : 0.9949104234527687
Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4880
         1.0       0.61      0.59      0.60        32

    accuracy                           0.99      4912
   macro avg       0.81      0.80      0.80      4912
weighted avg       0.99      0.99      0.99      4912



In [112]:
model  = model .fit(X_train_scaled, y_train)

importances =model.feature_importances_
importances

# We can sort the features by their importance.
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.18440061237719257, 'PER'),
 (0.13709008135843312, 'is_Center'),
 (0.13695379012553704, 'BLK'),
 (0.0716822910892883, 'FTA'),
 (0.060667410101976155, 'Tm_3rd'),
 (0.04994119259882496, 'DWS'),
 (0.044133977665195376, 'FT'),
 (0.04174602536628056, 'TOV'),
 (0.039514204447141064, 'RPG'),
 (0.03942428445946935, 'TOV%'),
 (0.03803465030122715, 'ORB'),
 (0.03321468453682746, 'BLK%'),
 (0.021816803316821398, 'TRB'),
 (0.016650463086935196, 'TRB%'),
 (0.012544661907172306, '2P'),
 (0.011977852801392144, 'FGA'),
 (0.011312416534648132, 'GS'),
 (0.011135660026294255, 'BPM'),
 (0.0096963570296984, 'DRB%'),
 (0.009544851451109362, 'AST'),
 (0.009544851451109362, '3PAr'),
 (0.008484312400986098, 'DBPM'),
 (0.0004885655664402371, 'WS/48'),
 (0.0, 'is_Guard'),
 (0.0, 'is_Forward'),
 (0.0, 'eFG%'),
 (0.0, 'Year_season_start_2016'),
 (0.0, 'Year_season_start_2015'),
 (0.0, 'Year_season_start_2014'),
 (0.0, 'Year_season_start_2013'),
 (0.0, 'Year_season_start_2012'),
 (0.0, 'Year_season_start_2011'),

In [113]:
###MACHINE LEARNING FOR GUARD

Season_Stats_ML_DF[:] = np.nan_to_num(Season_Stats_ML_DF)

#X = Season_Stats_ML_DF_loans.copy()
X = Season_Stats_ML_DF.drop("AllNBA_Guard", axis=1)
X.head()        
                    



,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,...,Year_season_start_2011,Year_season_start_2012,Year_season_start_2013,Year_season_start_2014,Year_season_start_2015,Year_season_start_2016,PPG,APG,RPG,SPG
0,24.0,22.0,0.0,163.0,8.4,0.472,0.238,0.476,5.4,20.9,...,0.0,0.0,0.0,0.0,0.0,1.0,2.181818,0.181818,0.590909,0.045455
1,32.0,65.0,0.0,894.0,9.5,0.507,0.427,0.133,2.3,6.3,...,0.0,0.0,0.0,0.0,0.0,1.0,4.953846,1.923077,0.138462,0.384615
2,21.0,80.0,72.0,2298.0,14.4,0.530,0.309,0.251,5.3,14.1,...,0.0,0.0,0.0,0.0,0.0,1.0,12.737500,1.875000,0.500000,0.800000
3,22.0,5.0,0.0,17.0,-2.2,0.102,0.500,0.500,0.0,19.5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.200000,0.600000,0.000000,0.000000
4,25.0,18.0,0.0,135.0,14.4,0.505,0.278,0.352,7.7,21.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.500000,0.388889,0.388889,0.444444


In [114]:
y = Season_Stats_ML_DF["AllNBA_Guard"].values.reshape(-1, 1)
y[:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [115]:
# Splitting into Train and Test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

from sklearn.preprocessing import StandardScaler
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn import tree
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)
predictions



# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)



In [116]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4845,3
Actual 1,5,59


Accuracy Score : 0.998371335504886
Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4848
         1.0       0.95      0.92      0.94        64

    accuracy                           1.00      4912
   macro avg       0.98      0.96      0.97      4912
weighted avg       1.00      1.00      1.00      4912



In [117]:
model  = model .fit(X_train_scaled, y_train)

importances =model.feature_importances_
importances

# We can sort the features by their importance.
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.35454923945672473, 'is_Guard'),
 (0.20448994975955698, 'OBPM'),
 (0.12687503983229448, 'FG'),
 (0.09934425237342207, 'Tm_2nd'),
 (0.06520294435267887, 'Tm_3rd'),
 (0.053183954775460375, 'PER'),
 (0.02948807890087942, 'Tm_1st'),
 (0.01590057093197373, 'DWS'),
 (0.010236359293918406, 'FTr'),
 (0.007982041773122854, 'TOV'),
 (0.00710858284299889, 'OWS'),
 (0.006303640374012259, 'ORB'),
 (0.005686866274399115, 'STL%'),
 (0.005118179646959201, 'GS'),
 (0.004265149705799335, 'eFG%'),
 (0.004265149705799335, '2P%'),
 (0.0, 'is_Forward'),
 (0.0, 'is_Center'),
 (0.0, 'Year_season_start_2016'),
 (0.0, 'Year_season_start_2015'),
 (0.0, 'Year_season_start_2014'),
 (0.0, 'Year_season_start_2013'),
 (0.0, 'Year_season_start_2012'),
 (0.0, 'Year_season_start_2011'),
 (0.0, 'Year_season_start_2010'),
 (0.0, 'Year_season_start_2009'),
 (0.0, 'Year_season_start_2008'),
 (0.0, 'Year_season_start_2007'),
 (0.0, 'Year_season_start_2006'),
 (0.0, 'Year_season_start_2005'),
 (0.0, 'Year_season_start_2004

In [118]:
####machine learning for forward

Season_Stats_ML_DF[:] = np.nan_to_num(Season_Stats_ML_DF)

#X = Season_Stats_ML_DF_loans.copy()
X = Season_Stats_ML_DF.drop("AllNBA_Forward", axis=1)
X.head()        
                    

y = Season_Stats_ML_DF["AllNBA_Forward"].values.reshape(-1, 1)
y[:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [119]:
# Splitting into Train and Test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

from sklearn.preprocessing import StandardScaler
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn import tree
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)
predictions



# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [120]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4843,8
Actual 1,6,55


Accuracy Score : 0.9971498371335505
Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4851
         1.0       0.87      0.90      0.89        61

    accuracy                           1.00      4912
   macro avg       0.94      0.95      0.94      4912
weighted avg       1.00      1.00      1.00      4912



In [121]:
model  = model .fit(X_train_scaled, y_train)

importances =model.feature_importances_
importances

# We can sort the features by their importance.
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.43053578579852564, 'is_Forward'),
 (0.20995400776501963, 'WS'),
 (0.09200766381273466, 'Tm_3rd'),
 (0.08459674436043521, 'Tm_2nd'),
 (0.05491762631970668, 'USG%'),
 (0.018551943279134456, 'BPM'),
 (0.018026005379057207, 'Tm_1st'),
 (0.01789347911255364, 'eFG%'),
 (0.016593481710347874, 'PF'),
 (0.009082747883558836, 'VORP'),
 (0.0062197077898283265, 'Tm_TOR'),
 (0.006181314531866432, 'Tm_DEN'),
 (0.005955039373239903, 'Year_season_start_2008'),
 (0.005904510324463421, 'FGA'),
 (0.005298269598742654, 'PER'),
 (0.004768442638868388, 'WS/48'),
 (0.004742353574227329, 'SPG'),
 (0.0031789617592455923, 'FTr'),
 (0.003084067378372583, 'Year_season_start_2005'),
 (0.0014843182557689593, 'Year_season_start_1997'),
 (0.0010235293543025457, 'Year_season_start_2001'),
 (0.0, 'is_Guard'),
 (0.0, 'is_Center'),
 (0.0, 'Year_season_start_2016'),
 (0.0, 'Year_season_start_2015'),
 (0.0, 'Year_season_start_2014'),
 (0.0, 'Year_season_start_2013'),
 (0.0, 'Year_season_start_2012'),
 (0.0, 'Year_seaso